In [1]:
# import packages
import pandas as pd
import datetime as dt
from datetime import date
from sqlalchemy import create_engine
# from config import password
import matplotlib.pyplot as plt 

In [2]:
crashes = pd.read_csv("crashes.csv")

In [3]:
crashes.columns

Index(['X', 'Y', 'OBJECTID', 'ACCIDENT_NO', 'ABS_CODE', 'ACCIDENT_STATUS',
       'ACCIDENT_DATE', 'ACCIDENT_TIME', 'ALCOHOLTIME', 'ACCIDENT_TYPE',
       'DAY_OF_WEEK', 'DCA_CODE', 'HIT_RUN_FLAG', 'LIGHT_CONDITION',
       'POLICE_ATTEND', 'ROAD_GEOMETRY', 'SEVERITY', 'SPEED_ZONE',
       'RUN_OFFROAD', 'NODE_ID', 'LONGITUDE', 'LATITUDE', 'NODE_TYPE',
       'LGA_NAME', 'REGION_NAME', 'VICGRID_X', 'VICGRID_Y', 'TOTAL_PERSONS',
       'INJ_OR_FATAL', 'FATALITY', 'SERIOUSINJURY', 'OTHERINJURY',
       'NONINJURED', 'MALES', 'FEMALES', 'BICYCLIST', 'PASSENGER', 'DRIVER',
       'PEDESTRIAN', 'PILLION', 'MOTORIST', 'UNKNOWN', 'PED_CYCLIST_5_12',
       'PED_CYCLIST_13_18', 'OLD_PEDESTRIAN', 'OLD_DRIVER', 'YOUNG_DRIVER',
       'ALCOHOL_RELATED', 'UNLICENCSED', 'NO_OF_VEHICLES', 'HEAVYVEHICLE',
       'PASSENGERVEHICLE', 'MOTORCYCLE', 'PUBLICVEHICLE', 'DEG_URBAN_NAME',
       'DEG_URBAN_ALL', 'LGA_NAME_ALL', 'REGION_NAME_ALL', 'SRNS', 'SRNS_ALL',
       'RMA', 'RMA_ALL', 'DIVIDED', 'DIVIDE

In [4]:
# add columns we want to remove to a list
remove_columns = ["OBJECTID","VICGRID_X","VICGRID_Y","SRNS","SRNS_ALL","DIVIDED_ALL","DIVIDED","REGION_NAME","LGA_NAME","DEG_URBAN_NAME","RMA_ALL","RMA","NODE_TYPE","NODE_ID","POLICE_ATTEND","ABS_CODE","ACCIDENT_STATUS","UNKNOWN","INJ_OR_FATAL","X","Y"]

In [5]:
# remove unwanted columns
crashes = crashes.drop(remove_columns,axis= 1)

In [6]:
# check if columns have been removed
crashes.columns

Index(['ACCIDENT_NO', 'ACCIDENT_DATE', 'ACCIDENT_TIME', 'ALCOHOLTIME',
       'ACCIDENT_TYPE', 'DAY_OF_WEEK', 'DCA_CODE', 'HIT_RUN_FLAG',
       'LIGHT_CONDITION', 'ROAD_GEOMETRY', 'SEVERITY', 'SPEED_ZONE',
       'RUN_OFFROAD', 'LONGITUDE', 'LATITUDE', 'TOTAL_PERSONS', 'FATALITY',
       'SERIOUSINJURY', 'OTHERINJURY', 'NONINJURED', 'MALES', 'FEMALES',
       'BICYCLIST', 'PASSENGER', 'DRIVER', 'PEDESTRIAN', 'PILLION', 'MOTORIST',
       'PED_CYCLIST_5_12', 'PED_CYCLIST_13_18', 'OLD_PEDESTRIAN', 'OLD_DRIVER',
       'YOUNG_DRIVER', 'ALCOHOL_RELATED', 'UNLICENCSED', 'NO_OF_VEHICLES',
       'HEAVYVEHICLE', 'PASSENGERVEHICLE', 'MOTORCYCLE', 'PUBLICVEHICLE',
       'DEG_URBAN_ALL', 'LGA_NAME_ALL', 'REGION_NAME_ALL', 'STAT_DIV_NAME'],
      dtype='object')

In [7]:
# check for NA values
crashes.isna().sum()[crashes.isna().sum()>0]

DAY_OF_WEEK         2899
NO_OF_VEHICLES         8
HEAVYVEHICLE           8
PASSENGERVEHICLE       8
MOTORCYCLE             8
PUBLICVEHICLE          8
REGION_NAME_ALL        1
STAT_DIV_NAME          2
dtype: int64

In [8]:
# check for blank data
crashes[crashes==" "].sum()

ACCIDENT_NO          0.0
ACCIDENT_DATE        0.0
ACCIDENT_TIME        0.0
ALCOHOLTIME          0.0
ACCIDENT_TYPE        0.0
DAY_OF_WEEK          0.0
DCA_CODE             0.0
HIT_RUN_FLAG         0.0
LIGHT_CONDITION      0.0
ROAD_GEOMETRY        0.0
SEVERITY             0.0
SPEED_ZONE           0.0
RUN_OFFROAD          0.0
LONGITUDE            0.0
LATITUDE             0.0
TOTAL_PERSONS        0.0
FATALITY             0.0
SERIOUSINJURY        0.0
OTHERINJURY          0.0
NONINJURED           0.0
MALES                0.0
FEMALES              0.0
BICYCLIST            0.0
PASSENGER            0.0
DRIVER               0.0
PEDESTRIAN           0.0
PILLION              0.0
MOTORIST             0.0
PED_CYCLIST_5_12     0.0
PED_CYCLIST_13_18    0.0
OLD_PEDESTRIAN       0.0
OLD_DRIVER           0.0
YOUNG_DRIVER         0.0
ALCOHOL_RELATED      0.0
UNLICENCSED          0.0
NO_OF_VEHICLES       0.0
HEAVYVEHICLE         0.0
PASSENGERVEHICLE     0.0
MOTORCYCLE           0.0
PUBLICVEHICLE        0.0


In [9]:
# check for duplicates
crashes["ACCIDENT_NO"].value_counts()

T20190010526    1
T20180015672    1
T20180019344    1
T20190024904    1
T20160027242    1
               ..
T20180011738    1
T20200002487    1
T20200009888    1
T20160026415    1
T20160022673    1
Name: ACCIDENT_NO, Length: 75320, dtype: int64

In [10]:
crashes["ACCIDENT_DATE"].value_counts()

2015/12/09 00:00:00+00    76
2016/04/29 00:00:00+00    75
2016/03/10 00:00:00+00    75
2015/12/04 00:00:00+00    73
2017/04/21 00:00:00+00    72
                          ..
2020/04/20 00:00:00+00    13
2020/04/13 00:00:00+00    12
2020/04/14 00:00:00+00    11
2020/04/26 00:00:00+00    11
2020/04/17 00:00:00+00    10
Name: ACCIDENT_DATE, Length: 1827, dtype: int64

In [11]:
crashes.columns

Index(['ACCIDENT_NO', 'ACCIDENT_DATE', 'ACCIDENT_TIME', 'ALCOHOLTIME',
       'ACCIDENT_TYPE', 'DAY_OF_WEEK', 'DCA_CODE', 'HIT_RUN_FLAG',
       'LIGHT_CONDITION', 'ROAD_GEOMETRY', 'SEVERITY', 'SPEED_ZONE',
       'RUN_OFFROAD', 'LONGITUDE', 'LATITUDE', 'TOTAL_PERSONS', 'FATALITY',
       'SERIOUSINJURY', 'OTHERINJURY', 'NONINJURED', 'MALES', 'FEMALES',
       'BICYCLIST', 'PASSENGER', 'DRIVER', 'PEDESTRIAN', 'PILLION', 'MOTORIST',
       'PED_CYCLIST_5_12', 'PED_CYCLIST_13_18', 'OLD_PEDESTRIAN', 'OLD_DRIVER',
       'YOUNG_DRIVER', 'ALCOHOL_RELATED', 'UNLICENCSED', 'NO_OF_VEHICLES',
       'HEAVYVEHICLE', 'PASSENGERVEHICLE', 'MOTORCYCLE', 'PUBLICVEHICLE',
       'DEG_URBAN_ALL', 'LGA_NAME_ALL', 'REGION_NAME_ALL', 'STAT_DIV_NAME'],
      dtype='object')

In [12]:
#drop all NA's
crashes = crashes.dropna(axis = 0)

In [13]:
crashes.isna().sum()

ACCIDENT_NO          0
ACCIDENT_DATE        0
ACCIDENT_TIME        0
ALCOHOLTIME          0
ACCIDENT_TYPE        0
DAY_OF_WEEK          0
DCA_CODE             0
HIT_RUN_FLAG         0
LIGHT_CONDITION      0
ROAD_GEOMETRY        0
SEVERITY             0
SPEED_ZONE           0
RUN_OFFROAD          0
LONGITUDE            0
LATITUDE             0
TOTAL_PERSONS        0
FATALITY             0
SERIOUSINJURY        0
OTHERINJURY          0
NONINJURED           0
MALES                0
FEMALES              0
BICYCLIST            0
PASSENGER            0
DRIVER               0
PEDESTRIAN           0
PILLION              0
MOTORIST             0
PED_CYCLIST_5_12     0
PED_CYCLIST_13_18    0
OLD_PEDESTRIAN       0
OLD_DRIVER           0
YOUNG_DRIVER         0
ALCOHOL_RELATED      0
UNLICENCSED          0
NO_OF_VEHICLES       0
HEAVYVEHICLE         0
PASSENGERVEHICLE     0
MOTORCYCLE           0
PUBLICVEHICLE        0
DEG_URBAN_ALL        0
LGA_NAME_ALL         0
REGION_NAME_ALL      0
STAT_DIV_NA

In [ ]:
# load data
lga = pd.read_csv("lga.csv")

In [ ]:
# find most recent population, and create dataframe with only the columns we need
lga = lga.rename(columns= {"Value": "Population Size"})
lga_2018 = lga[lga["Time"]==2018]
lga_2018 = lga_2018.iloc[2:,]
lga_2018 = lga_2018.iloc[:82-3,]
lga_2 = lga_2018[["Region","Population Size"]]

In [ ]:
lga_2[['LGA','letter','value']] = lga_2.Region.str.split(" ",expand=True,)
final_lga = lga_2.drop(['Region','letter','value'],axis=1)
final_lga = final_lga.reset_index()
final_lga= final_lga.drop("index",axis=1)
final_lga

In [ ]:
# accident table

In [15]:
# create accident light condition id column
crashes["accident_light_condition_ID"] = (crashes["LIGHT_CONDITION"].astype("category").cat.codes+1)

In [16]:
# Accident light condition table
accident_light_cond = pd.DataFrame(crashes["LIGHT_CONDITION"].unique())
accident_light_cond = accident_light_cond.rename(columns = {0:"LIGHT_CONDITION"})
accident_light_cond["ACCIDENT_LIGHT_CONDITION_ID"] = accident_light_cond["LIGHT_CONDITION"].astype("category").cat.codes+1
accident_light_cond = accident_light_cond.rename(columns={'LIGHT_CONDITION': 'ACCIDENT_LIGHT_CONDITION_DESCRIPTION'})


In [17]:
# change order of columns
accident_light_cond.insert(0,'ACCIDENT_LIGHT_CONDITION_ID',accident_light_cond.pop('ACCIDENT_LIGHT_CONDITION_ID'))

In [18]:
accident_light_cond

,ACCIDENT_LIGHT_CONDITION_ID,ACCIDENT_LIGHT_CONDITION_DESCRIPTION
0,6,Dusk/Dawn
1,5,Day
2,1,Dark No street lights
3,3,Dark Street lights on
4,7,Unk.
5,4,Dark Street lights unknown
6,2,Dark Street lights off


In [19]:
# create SEVERITY_ID column
crashes["accident_severity_ID"] = (crashes["SEVERITY"].astype("category").cat.codes+1)

In [20]:
# accident severity
severity = pd.DataFrame(crashes["SEVERITY"].unique())
severity = severity.rename(columns = {0:"accident_severity_description"})
severity["ACCIDENT_SEVERITY_ID"] = severity["accident_severity_description"].astype("category").cat.codes+1
severity = severity.rename(columns={'accident_severity_description': 'ACCIDENT_SEVERITY_DESCRIPTION'})

In [21]:
# change order of columns
severity.insert(0,'ACCIDENT_SEVERITY_ID',severity.pop('ACCIDENT_SEVERITY_ID'))

In [22]:
severity

,ACCIDENT_SEVERITY_ID,ACCIDENT_SEVERITY_DESCRIPTION
0,4,Serious injury accident
1,3,Other injury accident
2,1,Fatal accident
3,2,Non injury accident


In [23]:
# create ROAD_ID column
crashes["Road_ID"] = (crashes["ROAD_GEOMETRY"].astype("category").cat.codes+1)

In [24]:
#Road Table
road = pd.DataFrame(crashes["ROAD_GEOMETRY"].unique())
road = road.rename(columns = {0:"ROAD_GEOMETRY"})
road["ROAD_ID"] = road["ROAD_GEOMETRY"].astype("category").cat.codes+1

In [25]:
# change order of columns
road.insert(0,'ROAD_ID',road.pop('ROAD_ID'))

In [26]:
road

,ROAD_ID,ROAD_GEOMETRY
0,4,Not at intersection
1,1,Cross intersection
2,7,T intersection
3,5,Private property
4,8,Unknown
5,3,Multiple intersection
6,9,Y intersection
7,2,Dead end
8,6,Road closure


In [27]:
# ROAD SPEED Table
road_speed = pd.DataFrame(crashes["SPEED_ZONE"].unique())
road_speed = road_speed.rename(columns = {0:"ROAD_SPEED"})
road_speed = road_speed.rename(columns={'ROAD_SPEED': 'ROAD_SPEED_ZONE'})

In [28]:
road_speed

,ROAD_SPEED_ZONE
0,100 km/hr
1,60 km/hr
2,80 km/hr
3,50 km/hr
4,Not known
5,Camping grounds or off road
6,110 km/hr
7,70 km/hr
8,40 km/hr
9,30km/hr


In [ ]:
# create ACCIDENT_TYPE column
crashes["ACCIDENT_TYPE_ID"] = (crashes["ACCIDENT_TYPE"].astype("category").cat.codes+1)

In [ ]:
crashes["ACCIDENT_TYPE"]

In [34]:
# accident type
accident_type = pd.DataFrame(crashes["ACCIDENT_TYPE"].unique())
accident_type = accident_type.rename(columns = {0:"ACCIDENT_TYPE_DESCRIPTION"})
accident_type["ACCIDENT_TYPE_ID"] = accident_type["ACCIDENT_TYPE_DESCRIPTION"].astype("category").cat.codes+1


In [37]:
# change order of columns
accident_type.insert(0,'ACCIDENT_TYPE_ID',accident_type.pop('ACCIDENT_TYPE_ID'))

In [39]:
accident_type

,ACCIDENT_TYPE_ID,ACCIDENT_TYPE_DESCRIPTION
0,2,Collision with vehicle
1,1,Collision with a fixed object
2,8,Vehicle overturned (no collision)
3,6,Struck Pedestrian
4,7,Struck animal
5,4,No collision and no object struck
6,9,collision with some other object
7,3,Fall from or in moving vehicle
8,5,Other accident


In [47]:
# state div
state_div = pd.DataFrame(crashes["STAT_DIV_NAME"].unique())
state_div = state_div.rename(columns = {0:"STAT_DIV_NAME"})
state_div["STATE_DIV_ID"] = state_div["STAT_DIV_NAME"].astype("category").cat.codes+1
state_div = state_div.rename(columns={'STAT_DIV_NAME': 'STATE_DIV_NAME'})

In [48]:
# change order of columns
state_div.insert(0,'STATE_DIV_ID',state_div.pop('STATE_DIV_ID'))

In [49]:
state_div

,STATE_DIV_ID,STATE_DIV_NAME
0,2,Metro
1,1,Country


In [56]:
# Degree_urban
urban = pd.DataFrame(crashes["DEG_URBAN_ALL"].unique())
urban = urban.rename(columns = {0:"DEG_URBAN_DESCRIPTION"})
urban["DEGREE_URBAN_ID"] = urban["DEG_URBAN_DESCRIPTION"].astype("category").cat.codes+1
urban["STATE_DIV_ID"] = crashes["STAT_DIV_NAME"].astype("category").cat.codes+1
urban = urban.rename(columns={'DEG_URBAN_DESCRIPTION': 'DEGREE_URBAN_DESCRIPTION'})

In [57]:
# change order of columns
urban.insert(0,'DEGREE_URBAN_ID',urban.pop('DEGREE_URBAN_ID'))

In [59]:
urban

,DEGREE_URBAN_ID,DEGREE_URBAN_DESCRIPTION,STATE_DIV_ID
0,5,MELB_URBAN,2
1,9,RURAL_VICTORIA,2
2,15,SMALL_CITIES,2
3,21,TOWNS,2
4,1,LARGE_PROVINCIAL_CITIES,2
5,6,"MELB_URBAN,MELBOURNE_CBD",1
6,3,MELBOURNE_CBD,2
7,17,SMALL_TOWNS,2
8,14,"RURAL_VICTORIA,TOWNS",2
9,7,"MELB_URBAN,RURAL_VICTORIA",2


In [64]:
# Region table
region = pd.DataFrame(crashes["REGION_NAME_ALL"].unique())
region = region.rename(columns = {0:"REGION_NAME"})
region["REGION_ID"] = region["REGION_NAME"].astype("category").cat.codes+1
region["DEGREE_URBAN_ID"] = crashes["DEG_URBAN_ALL"].astype("category").cat.codes+1

In [65]:
# change order of columns
region.insert(0,'REGION_ID',region.pop('REGION_ID'))

In [66]:
region

,REGION_ID,REGION_NAME,DEGREE_URBAN_ID
0,5,METROPOLITAN SOUTH EAST REGION,5
1,4,METROPOLITAN NORTH WEST REGION,5
2,13,SOUTH WESTERN REGION,5
3,14,WESTERN REGION,5
4,1,"EASTERN REGION,EASTERN REGION",5
5,11,NORTHERN REGION,9
6,7,NORTH EASTERN REGION,5
7,16,"WESTERN REGION,SOUTH WESTERN REGION",5
8,6,"METROPOLITAN SOUTH EAST REGION,METROPOLITAN NO...",15
9,10,"NORTH EASTERN REGION,NORTHERN REGION",5


In [67]:
# create LGA_ID column
crashes["LGA_ID"] = (crashes["LGA_NAME_ALL"].astype("category").cat.codes+1)

In [ ]:
# Need to check if the below two tables work or not

In [ ]:
# LGA table
LGA = pd.DataFrame(final_lga["LGA"].unique())
LGA = LGA.rename(columns = {0:"LGA"})
LGA["LGA_ID"] = LGA["LGA"].astype("category").cat.codes+1


In [ ]:
# Population table
population = pd.DataFrame(final_lga)
population["LGA_ID"] = population["LGA"].astype("category").cat.codes+1


In [86]:
# accident table
accident_column_required = ["ACCIDENT_NO","ACCIDENT_DATE","ACCIDENT_TIME","ALCOHOLTIME","DAY_OF_WEEK","DCA_CODE","HIT_RUN_FLAG","RUN_OFFROAD","LATITUDE","LONGITUDE","TOTAL_PERSONS","NO_OF_VEHICLES","ALCOHOL_RELATED","UNLICENCSED"]

In [87]:
accident = crashes[accident_column_required]

In [89]:
accident["ACCIDENT_DATETIME"] = accident["ACCIDENT_DATE"] + accident["ACCIDENT_TIME"]

E:\ANACONDA3\envs\pythonData\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [93]:
del accident["ACCIDENT_DATE"]

In [94]:
del accident["ACCIDENT_TIME"]

In [99]:
accident = accident.rename(columns={'ALCOHOLTIME':'ACCIDENT_ALCHOL_TIME', 'DAY_OF_WEEK':'ACCIDENT_DAY_OF_WEEK', 'DCA_CODE':'ACCIDENT_DCA_CODE', 'HIT_RUN_FLAG':'ACCIDENT_HIT_RUN_FLAG',
'RUN_OFFROAD':'ACCIDENT_RUN_OFFROAD', 'LATITUDE':'ACCIDENT_LATITUDE', 'LONGITUDE':'ACCIDENT_LONGTITUDE', 'TOTAL_PERSONS':'ACCIDENT_TOTAL_PEOPLE_INVOLVED_COUNT',
'NO_OF_VEHICLES':'ACCIDENT_TOTAL_VEHICLE_INVOLVED_COUNT', 'ALCOHOL_RELATED':'ACCIDENT_ALCOHOL_RELATED', 'UNLICENCSED':'ACCIDENT_WITHOUT_LICENSE'})

In [101]:
accident["ACCIDENT_TYPE_ID"] = crashes["ACCIDENT_TYPE"].astype("category").cat.codes+1

In [103]:
accident["ACCIDENT_LIGHT_CONDITION_ID"] = crashes["LIGHT_CONDITION"].astype("category").cat.codes+1

In [105]:
accident["ACCIDENT_SEVERITY_ID"] = crashes["SEVERITY"].astype("category").cat.codes+1

In [106]:
accident["ROAD_ID"] = crashes["ROAD_GEOMETRY"].astype("category").cat.codes+1

In [ ]:
accident["LGA_ID"] = LGA["LGA"].astype("category").cat.codes+1


In [108]:
accident["ROAD_SPEED_ZONE"] = crashes["SPEED_ZONE"]

In [109]:
accident.head()

,ACCIDENT_NO,ACCIDENT_ALCHOL_TIME,ACCIDENT_DAY_OF_WEEK,ACCIDENT_DCA_CODE,ACCIDENT_HIT_RUN_FLAG,ACCIDENT_RUN_OFFROAD,ACCIDENT_LATITUDE,ACCIDENT_LONGTITUDE,ACCIDENT_TOTAL_PEOPLE_INVOLVED_COUNT,ACCIDENT_TOTAL_VEHICLE_INVOLVED_COUNT,ACCIDENT_ALCOHOL_RELATED,ACCIDENT_WITHOUT_LICENSE,ACCIDENT_DATETIME,ACCIDENT_TYPE_ID,ACCIDENT_LIGHT_CONDITION_ID,ACCIDENT_SEVERITY_ID,ROAD_ID,ROAD_SPEED_ZONE
0,T20150013821,No,Wednesday,REAR END(VEHICLES IN SAME LANE),No,No,-37.971209,145.237220,4,4.0,No,0,2015/07/01 00:00:00+001899/12/30 07:00:00+00,2,6,4,4,100 km/hr
1,T20150013832,No,Wednesday,CROSS TRAFFIC(INTERSECTIONS ONLY),No,No,-38.065277,145.298911,2,2.0,No,0,2015/07/01 00:00:00+001899/12/30 09:55:00+00,2,5,4,1,60 km/hr
2,T20150013882,Yes,Wednesday,OFF RIGHT BEND INTO OBJECT/PARKED VEHICLE,No,Yes,-37.894931,145.441219,1,1.0,No,0,2015/07/01 00:00:00+001899/12/30 21:50:00+00,1,1,4,4,60 km/hr
3,T20150013886,Yes,Wednesday,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICLE,No,Yes,-37.993356,145.188348,1,1.0,No,0,2015/07/01 00:00:00+001899/12/30 06:00:00+00,1,3,4,7,80 km/hr
4,T20150013922,No,Thursday,VEHICLE STRIKES ANOTHER VEH WHILE EMERGING FRO...,No,No,-37.775443,144.831010,2,2.0,No,0,2015/07/02 00:00:00+001899/12/30 15:50:00+00,2,5,3,7,50 km/hr


In [71]:
# people involved
wanted_columns = ["ACCIDENT_NO","FATALITY","SERIOUSINJURY","OTHERINJURY","NONINJURED","MALES","FEMALES","BICYCLIST",'PASSENGER', 'DRIVER', 'PEDESTRIAN', 'PILLION', 'MOTORIST','PED_CYCLIST_5_12', 'PED_CYCLIST_13_18', 'OLD_PEDESTRIAN', 'OLD_DRIVER','YOUNG_DRIVER']
people_involved =crashes[wanted_columns]

In [72]:
people_involved = people_involved.rename(columns={'FATALITY': 'ACCIDENT_FATALITY_COUNT', 'SERIOUSINJURY': 'ACCIDENT_SERIOUS_INJURY_COUNT', 
'OTHERINJURY':'ACCIDENT_OTHER_INJURY_COUNT', 'NONINJURED':'ACCIDENT_NON_INJURY_COUNT', 'MALES':'ACCIDENT_MALE_COUNT', 'FEMALES':'ACCIDENT_FEMALE_COUNT', 
'BICYCLIST':'ACCIDENT_BICYCLIST_COUNT', 'PASSENGER':'ACCIDENT_PASSENGER_COUNT', 'DRIVER':'ACCIDENT_DRIVER_COUNT', 'PEDESTRIAN':'ACCIDENT_PEDESTRIAN_COUNT', 
'PILLION':'ACCIDENT_PILLION_COUNT', 'MOTORIST':'ACCIDENT_MOTORIST_COUNT', 'PED_CYCLIST_5_12':'ACCIDENT_PED_CYCLIST_5TO12_COUNT', 
'PED_CYCLIST_13_18':'ACCIDENT_PED_CYCLIST_13TO18_COUNT', 'OLD_PEDESTRIAN':'ACCIDENT_OLD_PEDESTRIAN_COUNT', 'OLD_DRIVER':'ACCIDENT_OLD_DRIVER_COUNT', 
'YOUNG_DRIVER':'ACCIDENT_YOUNG_DRIVER_COUNT'})

In [73]:
people_involved

,ACCIDENT_NO,ACCIDENT_FATALITY_COUNT,ACCIDENT_SERIOUS_INJURY_COUNT,ACCIDENT_OTHER_INJURY_COUNT,ACCIDENT_NON_INJURY_COUNT,ACCIDENT_MALE_COUNT,ACCIDENT_FEMALE_COUNT,ACCIDENT_BICYCLIST_COUNT,ACCIDENT_PASSENGER_COUNT,ACCIDENT_DRIVER_COUNT,ACCIDENT_PEDESTRIAN_COUNT,ACCIDENT_PILLION_COUNT,ACCIDENT_MOTORIST_COUNT,ACCIDENT_PED_CYCLIST_5TO12_COUNT,ACCIDENT_PED_CYCLIST_13TO18_COUNT,ACCIDENT_OLD_PEDESTRIAN_COUNT,ACCIDENT_OLD_DRIVER_COUNT,ACCIDENT_YOUNG_DRIVER_COUNT
0,T20150013821,0,1,0,3,2,2,0,0,4,0,0,0,0,0,0,0,2
1,T20150013832,0,1,0,1,2,0,0,0,2,0,0,0,0,0,0,1,0
2,T20150013882,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0
3,T20150013886,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0
4,T20150013922,0,0,1,1,1,1,0,0,2,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75315,T20210004831,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0
75316,T20210005921,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
75317,T20210009491,0,0,2,0,1,1,0,0,2,0,0,0,0,0,0,0,1
75318,T20210017186,0,0,1,1,1,1,0,0,2,0,0,0,0,0,0,0,0


In [76]:
# vehicle_involed
col = ["ACCIDENT_NO",'HEAVYVEHICLE', 'PASSENGERVEHICLE', 'MOTORCYCLE', 'PUBLICVEHICLE']
vehicles_involed = crashes[col]

In [78]:
vehicles_involed = vehicles_involed.rename(columns={'HEAVYVEHICLE':'ACCIDENT_HEAVY_VEHICLE_COUNT', 'PASSENGERVEHICLE':'ACCIDENT_PASSENGER_VEHICLE_COUNT',
'MOTORCYCLE':'ACCIDENT_MOTORCYCLE_COUNT', 'PUBLICVEHICLE':'ACCIDENT_PUBLIC_VEHICLE_COUNT'})

In [79]:
vehicles_involed


,ACCIDENT_NO,ACCIDENT_HEAVY_VEHICLE_COUNT,ACCIDENT_PASSENGER_VEHICLE_COUNT,ACCIDENT_MOTORCYCLE_COUNT,ACCIDENT_PUBLIC_VEHICLE_COUNT
0,T20150013821,0.0,4.0,0.0,0.0
1,T20150013832,0.0,1.0,0.0,1.0
2,T20150013882,0.0,1.0,0.0,0.0
3,T20150013886,0.0,0.0,1.0,0.0
4,T20150013922,0.0,2.0,0.0,0.0
...,...,...,...,...,...
75315,T20210004831,0.0,0.0,1.0,0.0
75316,T20210005921,0.0,0.0,1.0,0.0
75317,T20210009491,0.0,2.0,0.0,0.0
75318,T20210017186,0.0,2.0,0.0,0.0


In [80]:
# create a connection
connection_string = f"postgres:{PASSWORD}@localhost:5432/Road_crash_DB"
engine = create_engine(f'postgresql://{connection_string}')


In [81]:
# look at table names
engine.table_names()

E:\ANACONDA3\envs\pythonData\lib\site-packages\ipykernel_launcher.py:2: SADeprecationWarning: The Engine.table_names() method is deprecated and will be removed in a future release.  Please refer to Inspector.get_table_names(). (deprecated since: 1.4)
  


['STATE_DIV',
 'DEGREE_URBAN',
 'REGION',
 'LGA',
 'ACCIDENT_TYPE',
 'ACCIDENT',
 'ACCIDENT_LIGHT_CONDITION',
 'ACCIDENT_SEVERITY',
 'ROAD',
 'ROAD_SPEED',
 'ACCIDENT_PERSON_INVOLVED_COUNT',
 'ACCIDENT_VEHICLE_INVOLVED_COUNT']

In [110]:
accident_light_cond.to_sql(name='ACCIDENT_LIGHT_CONDITION', con=engine, if_exists='append', index=False)

In [114]:
accident_type.to_sql(name='ACCIDENT_TYPE', con=engine, if_exists='append', index=False)

In [111]:
road.to_sql(name='ROAD', con=engine, if_exists='append', index=False)

In [112]:
road_speed.to_sql(name='ROAD_SPEED', con=engine, if_exists='append', index=False)

In [113]:
# try insert into database
severity.to_sql(name='ACCIDENT_SEVERITY', con=engine, if_exists='append', index=False)

In [115]:
state_div.to_sql(name='STATE_DIV', con=engine, if_exists='append', index=False)

In [116]:
urban.to_sql(name='DEGREE_URBAN', con=engine, if_exists='append', index=False)

In [117]:
region.to_sql(name='REGION', con=engine, if_exists='append', index=False)

In [118]:
# Need to insert LGA table

In [119]:
# Need to insert ACCIDENT table

In [ ]:
# Need to insert ACCIDENT_PERSON_INVOLVED_COUNT table

In [ ]:
# Need to insert ACCIDENT_VEHICLE_INVOLVED_COUNT table
vehicles_involed.to_sql(name='ACCIDENT_VEHICLE_INVOLVED_COUNT', con=engine, if_exists='append', index=False)

In [ ]:
# group by accidents per day
a = crashes.groupby("DAY_OF_WEEK").count()["ACCIDENT_NO"]